<a href="https://colab.research.google.com/github/dasjyotishka/Aspect-Term-Polarity-Classification-in-Sentiment-Analysis/blob/main/Methane_Acc_84_AUC_89_Epoch_49.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [52]:
from google.colab import drive 
drive.mount('/content/drive',)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt

In [54]:
# Define hyperparameters
#batch_size = 64
batch_size = 32
learning_rate = 0.001
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset class
class PlumeDataset(Dataset):
    def __init__(self, plume_dir, no_plume_dir, transform=None):
        self.plume_images = [os.path.join(plume_dir, img) for img in os.listdir(plume_dir)]
        self.no_plume_images = [os.path.join(no_plume_dir, img) for img in os.listdir(no_plume_dir)]
        self.images = self.plume_images + self.no_plume_images
        self.targets = [1] * len(self.plume_images) + [0] * len(self.no_plume_images)
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('L')
        target = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        return image, target

# Define data transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])




#Dataloaders

The original dataset contained 214 images containing a methane plume, and 214 images not containing a methane plume. Now, to train our model we created four additional folders. The plume_training folders contain 192 images containg a methane plume and the no_plume_training folder has 192 images not containing a methane plume. The plume_testing folders contain 22 images containg a methane plume and the no_plume_testing folder contains 22 images not containing a methane plume selected manually at random.

In [55]:
# Create datasets and dataloaders
train_dataset = PlumeDataset('/content/drive/MyDrive/QB/plume_training', '/content/drive/MyDrive/QB/no_plume_training', transform=train_transforms)
test_dataset = PlumeDataset('/content/drive/MyDrive/QB/plume_testing', '/content/drive/MyDrive/QB/no_plume_testing', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define CNN model
class PlumeCNN(nn.Module):
    def __init__(self):
        super(PlumeCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(self.bn1(nn.functional.relu(self.conv1(x))))
        x = self.pool(self.bn2(nn.functional.relu(self.conv2(x))))
        x = x.view(-1, 64 * 16 * 16)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = nn.functional.sigmoid(self.fc2(x))
        return x




In [56]:
# Initialize model and optimizer
model = PlumeCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [57]:
def evaluate(model, loader):
    model.eval()
    targets = []
    #Obtain a list of prediction scores. If the prediction score >= 0.5, it means the image contains a plume, else not
    predictions = []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images).squeeze()
            predictions += outputs.cpu().tolist()
            targets += labels.cpu().tolist()
    #accuracy = np.mean(np.array(predictions) > 0.5) * 100
    accuracy = accuracy_score(targets, [round(p) for p in predictions])
    auc_score = roc_auc_score(targets, predictions) * 100
    return accuracy, auc_score


#Training the model and get test accuracy and AUC score

In [58]:
# Train the model
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_dataset)
    test_accuracy, test_auc_score = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.2f}, Test AUC: {test_auc_score:.2f}%")


Epoch 1/50, Train Loss: 1.2479, Test Accuracy: 0.64, Test AUC: 68.80%
Epoch 2/50, Train Loss: 0.8618, Test Accuracy: 0.55, Test AUC: 59.09%
Epoch 3/50, Train Loss: 0.7389, Test Accuracy: 0.64, Test AUC: 76.86%
Epoch 4/50, Train Loss: 0.6053, Test Accuracy: 0.68, Test AUC: 75.62%
Epoch 5/50, Train Loss: 0.6163, Test Accuracy: 0.61, Test AUC: 73.55%
Epoch 6/50, Train Loss: 0.5800, Test Accuracy: 0.61, Test AUC: 70.25%
Epoch 7/50, Train Loss: 0.6365, Test Accuracy: 0.59, Test AUC: 70.45%
Epoch 8/50, Train Loss: 0.5745, Test Accuracy: 0.66, Test AUC: 74.17%
Epoch 9/50, Train Loss: 0.5625, Test Accuracy: 0.61, Test AUC: 71.69%
Epoch 10/50, Train Loss: 0.5535, Test Accuracy: 0.66, Test AUC: 77.07%
Epoch 11/50, Train Loss: 0.5214, Test Accuracy: 0.75, Test AUC: 83.26%
Epoch 12/50, Train Loss: 0.5227, Test Accuracy: 0.75, Test AUC: 79.96%
Epoch 13/50, Train Loss: 0.5364, Test Accuracy: 0.64, Test AUC: 76.24%
Epoch 14/50, Train Loss: 0.4989, Test Accuracy: 0.61, Test AUC: 81.40%
Epoch 15/50, Tr

In [59]:
best_model_path = "/content/drive/MyDrive/QB/models/model_Epoch_50_AUC_90.pt"
torch.save(model.state_dict(), best_model_path)

#Upload one image and get the classification score

In [61]:
from torchvision import transforms
from PIL import Image
import torch

# Define the image transform
last_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Load the saved model
model = PlumeCNN()
model.load_state_dict(torch.load("/content/drive/MyDrive/QB/models/model_Epoch_50_AUC_90.pt"))
model.eval()

# Load the input image
image = Image.open('/content/plume (208).tif')

# Apply the transform on the input image
image_tensor = last_transforms(image).float()

# Normalize the image tensor
image_tensor = transforms.Normalize(
    mean=[0.5],
    std=[0.5]
)(image_tensor)

# Add a batch dimension to the image tensor
image_tensor = image_tensor.unsqueeze(0)

# Pass the image tensor through the model to get the classification result
with torch.no_grad():

    #If sigmoid_output>=0.5, then positive, else negative
    output = model(image_tensor)

    print("Output: "+ str(output))

    sigmoid_output = torch.sigmoid(output).item()
    print("sigmoid_output: "+ str(sigmoid_output))

    #If prediction=1, then positive, else negative
    prediction = torch.sigmoid(output).round().item()
    print("prediction: "+ str(prediction))

# Print the predicted classification result
print(f'The image is classified as {"Positive" if prediction==1 else "Negative"}')


Output: tensor([[1.]])
sigmoid_output: 0.7310585975646973
prediction: 1.0
The image is classified as Positive
